In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip -q install geopandas libpysal esda splot mapclassify

In [ ]:
!pip install geopandas pyproj shapely fiona
!pip install libpysal esda splot mapclassify
!pip install matplotlib pandas numpy

# 1. Load the shapefiles

In [ ]:
import os
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data_dir = "your_data_path_here"  # Update this to your actual data directory

paths = {
    "original": os.path.join(data_dir, "original_points.shp"),
    "donut": os.path.join(data_dir, "moved_points_donut.shp"),
    "agdm": os.path.join(data_dir, "moved_points_agdm.shp"),
}

gdfs = {k: gpd.read_file(v) for k, v in paths.items()}

for name, gdf in gdfs.items():
    print(name, "rows =", len(gdf), "| CRS =", gdf.crs)
    print("columns:", list(gdf.columns))
    print(gdf.head(2), "\n")


# 2. Project data to meters (required for spatial distances)

In [ ]:
target_epsg = 25832

for name in gdfs:
    gdfs[name] = gdfs[name].to_crs(epsg=target_epsg)
    gdfs[name]["x"] = gdfs[name].geometry.x
    gdfs[name]["y"] = gdfs[name].geometry.y

[gdfs[name].crs for name in gdfs]

# 3. Plot: overlay of point locations (original vs masked)

In [ ]:
out_dir = "your_output_path_here"  # Update this to your desired output directory
fig_dir = os.path.join(out_dir, "figures")
tab_dir = os.path.join(out_dir, "tables")

os.makedirs(fig_dir, exist_ok=True)
os.makedirs(tab_dir, exist_ok=True)

fig, ax = plt.subplots(figsize=(7,7))
gdfs["original"].plot(ax=ax, markersize=1, alpha=0.4, label="original")
gdfs["donut"].plot(ax=ax, markersize=1, alpha=0.4, label="donut")
gdfs["agdm"].plot(ax=ax, markersize=1, alpha=0.4, label="agdm")

ax.set_title("Point locations: original vs Donut vs AGDM")
ax.set_axis_off()
plt.tight_layout()

overlay_path = os.path.join(fig_dir, "points_overlay.png")
plt.savefig(overlay_path, dpi=250)
plt.show()

overlay_path

# 4. Build spatial weights (k-nearest neighbors)

In [ ]:
from libpysal.weights import KNN

k_main = 8

def knn_weights(gdf, k=8):
    coords = np.column_stack([gdf["x"].to_numpy(), gdf["y"].to_numpy()])
    w = KNN.from_array(coords, k=k)
    w.transform = "R"  # row-standardized
    return w

weights = {name: knn_weights(gdfs[name], k_main) for name in gdfs}

# quick check
print("n =", weights["original"].n)
print("mean neighbors =", np.mean(list(weights["original"].cardinalities.values())))


# 5. Global Moran’s I + Geary’s C (sleep_diso)

In [ ]:
from esda.moran import Moran
from esda.geary import Geary

def global_stats(gdf, w, col="sleep_diso"):
    y = np.asarray(gdf[col], dtype=float)
    moran = Moran(y, w, permutations=999)
    geary = Geary(y, w, permutations=999)
    return {
        "moran_I": moran.I,
        "moran_p": moran.p_sim,
        "moran_z": moran.z_sim,
        "geary_C": geary.C,
        "geary_p": geary.p_sim,
        "geary_z": geary.z_sim,
    }

rows = []
for ds in ["original", "donut", "agdm"]:
    stats = global_stats(gdfs[ds], weights[ds], "sleep_diso")
    rows.append({"dataset": ds, **stats})

global_df = pd.DataFrame(rows)
global_df

In [ ]:
global_path = os.path.join(tab_dir, "global_autocorrelation_sleep.csv")
global_df.to_csv(global_path, index=False)
global_path

# 6. Sensitivity analysis: Moran’s I for multiple k values

In [ ]:
ks = [6, 8, 10, 12]

sens_rows = []
for k in ks:
    for ds in ["original", "donut", "agdm"]:
        w = knn_weights(gdfs[ds], k=k)
        y = np.asarray(gdfs[ds]["sleep_diso"], dtype=float)
        m = Moran(y, w, permutations=999)
        sens_rows.append({
            "dataset": ds,
            "k": k,
            "moran_I": m.I,
            "p": m.p_sim
        })

sens_df = pd.DataFrame(sens_rows)

# plot
fig, ax = plt.subplots(figsize=(7,4))
for ds in ["original", "donut", "agdm"]:
    d = sens_df[sens_df["dataset"] == ds].sort_values("k")
    ax.plot(d["k"], d["moran_I"], marker="o", label=ds)

ax.set_title("Sensitivity of Moran's I (sleep_diso) to k")
ax.set_xlabel("k nearest neighbors")
ax.set_ylabel("Moran's I")
ax.legend()
plt.tight_layout()

sens_plot_path = os.path.join(fig_dir, "sensitivity_moran_sleep.png")
plt.savefig(sens_plot_path, dpi=250)
plt.show()

sens_plot_path

In [ ]:
sens_table_path = os.path.join(tab_dir, "sensitivity_moran_sleep.csv")
sens_df.to_csv(sens_table_path, index=False)
sens_table_path

# 7. Local Moran (LISA) clusters

In [ ]:
from esda.moran import Moran_Local

def lisa_labels(local_moran, y, w):
    z = (y - y.mean()) / y.std()
    lag_z = w.sparse @ z
    q = local_moran.q
    sig = local_moran.p_sim < 0.05

    labels = np.full(len(y), "not significant", dtype=object)
    labels[(q == 1) & sig] = "high-high"
    labels[(q == 2) & sig] = "low-high"
    labels[(q == 3) & sig] = "low-low"
    labels[(q == 4) & sig] = "high-low"
    return labels

lisa_results = {}

for ds in ["original", "donut", "agdm"]:
    gdf = gdfs[ds].copy()
    y = np.asarray(gdf["sleep_diso"], dtype=float)
    w = weights[ds]

    ml = Moran_Local(y, w, permutations=999)

    gdf["lisa_I"] = ml.Is
    gdf["lisa_p"] = ml.p_sim
    gdf["lisa_cluster"] = lisa_labels(ml, y, w)

    lisa_results[ds] = gdf

# check counts
for ds in lisa_results:
    print(ds)
    print(lisa_results[ds]["lisa_cluster"].value_counts(), "\n")

# 8. Plot LISA cluster maps (single slide figure)

In [ ]:
def plot_clusters(gdf, title, ax):
    order = ["high-high", "low-low", "high-low", "low-high", "not significant"]
    gdf = gdf.copy()
    gdf["lisa_cluster"] = pd.Categorical(gdf["lisa_cluster"], categories=order, ordered=True)
    gdf.plot(column="lisa_cluster", ax=ax, markersize=3, legend=True)
    ax.set_title(title)
    ax.set_axis_off()

fig, axes = plt.subplots(1, 3, figsize=(15,5))
plot_clusters(lisa_results["original"], "Original", axes[0])
plot_clusters(lisa_results["donut"], "Donut masking", axes[1])
plot_clusters(lisa_results["agdm"], "AGDM masking", axes[2])

plt.tight_layout()
lisa_map_path = os.path.join(fig_dir, "lisa_clusters_triptych.png")
plt.savefig(lisa_map_path, dpi=250)
plt.show()

lisa_map_path


# 9. Quantitative similarity (original vs donut / original vs AGDM)

In [ ]:
from scipy.stats import spearmanr

def align_by_id(gdf_a, gdf_b):
    a = gdf_a[["id", "lisa_I", "lisa_p", "lisa_cluster"]].copy()
    b = gdf_b[["id", "lisa_I", "lisa_p", "lisa_cluster"]].copy()
    return a.merge(b, on="id", suffixes=("_orig", "_mask"))

def jaccard(a, b):
    inter = np.logical_and(a, b).sum()
    union = np.logical_or(a, b).sum()
    return inter / union if union > 0 else np.nan

orig = lisa_results["original"]

sim_rows = []
for masked in ["donut", "agdm"]:
    merged = align_by_id(orig, lisa_results[masked])

    rho, _ = spearmanr(merged["lisa_I_orig"], merged["lisa_I_mask"])

    sig_o = merged["lisa_p_orig"] < 0.05
    sig_m = merged["lisa_p_mask"] < 0.05
    jac_sig = jaccard(sig_o.to_numpy(), sig_m.to_numpy())

    either = (sig_o | sig_m).to_numpy()
    agree = (merged["lisa_cluster_orig"] == merged["lisa_cluster_mask"]).to_numpy()
    cluster_agree = (agree & either).sum() / either.sum()

    sim_rows.append({
        "masked_method": masked,
        "spearman_local_I": rho,
        "jaccard_significance": jac_sig,
        "cluster_agreement_on_sig": cluster_agree
    })

sim_df = pd.DataFrame(sim_rows)
sim_df


In [ ]:
sim_path = os.path.join(tab_dir, "lisa_similarity_sleep.csv")
sim_df.to_csv(sim_path, index=False)
sim_path
